# Dealing with Imbalanced Class Variable

In classification problems, sometimes we may encounter datasets that have class imbalance issue: a class value dominates the other class values. This poses significant challenges to the classification algorithms which work well with balanced class. This type of class imbalance problem can be found in areas like medical diagnosis, spam filtering, and fraud detection.

In this lab, we'll look at possible ways to handle an imbalanced class problem using a credit card data. Here each transaction is tagged as fradulent (1) or not fradulent (0). Our objective will be to correctly classify the minority class of fraudulent transactions.

Important Note: This guide will focus soley on addressing imbalanced classes and will not addressing other important machine learning steps including, but not limited to, feature selection or hyperparameter tuning.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, classification_report
from sklearn.metrics import fbeta_score

%matplotlib inline

In [2]:
# read in data
datapath = "./credit_card.csv"
df = pd.read_csv(datapath)

print(df.shape)

# show random 5 rows from this dataset
df.sample(n=5, replace=False)

(28480, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
337,157801.0,1.973538,-0.147425,-0.599246,1.088049,0.073275,0.383365,-0.451486,0.098268,0.906635,...,-0.204209,-0.500164,0.308062,-0.040861,-0.271705,-0.929683,0.054986,-0.021730,19.90,0
13361,163724.0,1.986017,-0.380205,-0.281325,0.467055,-0.813958,-0.708327,-0.568642,0.016825,1.493268,...,-0.194224,-0.453238,0.401922,-0.097008,-0.489213,-0.613725,0.030287,-0.035312,1.00,0
19862,166948.0,2.035861,-0.200976,-1.204585,0.238728,-0.035093,-0.732477,0.000754,-0.103041,0.476917,...,-0.270120,-0.740736,0.299643,-0.367221,-0.325599,0.201845,-0.082969,-0.075347,4.49,0
23330,169015.0,-0.124702,0.268215,1.223797,0.427078,-1.381094,0.018190,-0.826291,0.390685,-1.262411,...,0.020156,0.697320,-0.005881,0.132831,-0.855640,1.137074,0.113189,0.127342,24.99,0
14017,164032.0,2.082792,-0.991640,-1.127478,-0.670212,-0.447162,0.188657,-0.790566,0.068130,-0.333943,...,-0.344494,-0.535665,0.217781,0.078550,-0.329599,0.542222,-0.050758,-0.055737,45.00,0


In [3]:
# print the class 
df.Class.value_counts()

0    28431
1       49
Name: Class, dtype: int64

In [4]:
# print percentage of questions where target == 1
fraud_ratio = df[df.Class==1].shape[0] * 100 / df.shape[0]
fraud_ratio

0.1720505617977528

We can see we have a very imbalanced class - just 0.17% of our dataset belong to fradulent transactions!

This is a problem because many machine learning models are designed to maximize overall accuracy, which especially with imbalanced classes may not be the best metric to use. Classification accuracy is defined as the number of correct predictions divided by total predictions times 100. For example, if we simply predicted all transactions are not fraud, we would get a classification acuracy score of over 99%!

# Create Train and Test Sets
The training set is used to build and validate the model, while the test set is reserved for testing the model on unseen data.

In [5]:
# Prepare data for modeling
# Separate input features and target
y = df.Class
X = df.drop('Class', axis=1)

# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

In [6]:
y_train.value_counts()[1] * 100 / y_train.shape[0]

0.17322097378277154

In [7]:
y_test.value_counts()[1] * 100 / y_test.shape[0]

0.16853932584269662

# Baseline Models

In [8]:
# DummyClassifier to predict only target 0
dummy = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
dummy_pred = dummy.predict(X_test)

# checking unique labels
print('The predicted labels: ', np.unique(dummy_pred))

# checking accuracy
print('Accuracy score: ', accuracy_score(y_test, dummy_pred))

print(classification_report(y_test, dummy_pred))

The predicted labels:  [0]
Accuracy score:  0.998314606741573
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7108
           1       0.00      0.00      0.00        12

    accuracy                           1.00      7120
   macro avg       0.50      0.50      0.50      7120
weighted avg       1.00      1.00      1.00      7120



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The accuracy score for this Dummy Classifier is 99.8%, and this highy accuracy may mislead us. As the Dummy Classifier predicts only Class 0, it is clearly not a good option for our objective, which is correctly classifying fraudulent transactions.

Let's see how logistic regression performs on this dataset.

In [9]:
# Modeling the data as is
# Train model
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)
 
# Predict on training set
lr_pred = lr.predict(X_test)

In [10]:
# print confusion matrix
pd.DataFrame(confusion_matrix(y_test, lr_pred))

,0,1
0,7108,0
1,8,4


In [11]:
print(classification_report(y_test, lr_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7108
           1       1.00      0.33      0.50        12

    accuracy                           1.00      7120
   macro avg       1.00      0.67      0.75      7120
weighted avg       1.00      1.00      1.00      7120



In [12]:
# Checking accuracy to see the first few digits after decimal point
accuracy_score(y_test, lr_pred)

0.998876404494382

In [13]:
# Checking unique values
predictions = pd.DataFrame(lr_pred)
predictions[0].value_counts()

0    7116
1       4
Name: 0, dtype: int64

Logistic Regression outperformed the Dummy Classifier. We can see that it predicted 4 instances of class 1, so this is definitely an improvement. But can we do better?

Let's see if we can apply some techniques for dealing with class imbalance to improve these results. 

### 1. Change the evaluation measure

We should not use accuracy as a default measure for evaluating an imbalanced class. Metrics that can provide better insight include:
 
* **Confusion Matrix:** a table showing the number of correct and incorrect predictions
* **Precision:** the number of true positives divided by all positive predictions. Precision is also called Positive Predictive Value. It is a measure of a classifier's exactness. Low precision indicates a high number of false positives.
* **Recall:** the number of true positives divided by the number of positive values in the test data. Recall is also called Sensitivity or the True Positive Rate. It is a measure of a classifier's completeness. Low recall indicates a high number of false negatives.
* **F1-Score:** the weighted average of precision and recall
* **F-beta Score:** It's a generailzation of F1-score. F1-score give equal weights to precision and recall. In some case precision could be more important than recall and vice versa. Check this short [article](https://onlinehelp.explorance.com/blueml/Content/articles/getstarted/mlcalculations.htm?TocPath=Get%20started%7C_____3) on F-beta measure. 

If our main objective is to classifying the fraud cases the recall score (or F-2 score) can be considered a key metric to use for evaluating predictions.

In [14]:
# confusion matrix
pd.DataFrame(confusion_matrix(y_test, lr_pred))

,0,1
0,7108,0
1,8,4


In [15]:
# recall score
recall_score(y_test, lr_pred)

0.3333333333333333

In [16]:
# f1 score
f1_score(y_test, lr_pred)

0.5

In [17]:
# F2 Score
fbeta_score(y_test, lr_pred, beta=2)

0.3846153846153846

We have a very high accuracy score of 0.999 but a F1 score of only 0.752. And from the confusion matrix, we can see we are misclassifying several observations leading to a recall score of only 0.64.

### 2. Change the learner

For every machine learning problem, its a good rule of thumb to try a variety of algorithms, it can be beneficial with imbalanced datasets. Decision trees frequently perform well on imbalanced data. They work by learning a hierachy of if/else questions. This can force both classes to be addressed.

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
# train model
rfc = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

# predict on test set
rfc_pred = rfc.predict(X_test)

print(classification_report(y_test, rfc_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7108
           1       0.92      0.92      0.92        12

    accuracy                           1.00      7120
   macro avg       0.96      0.96      0.96      7120
weighted avg       1.00      1.00      1.00      7120



In [20]:
# confusion matrix
pd.DataFrame(confusion_matrix(y_test, rfc_pred))

,0,1
0,7107,1
1,1,11


It seems RandomForest was able to overcome the class imbalance problem for this dataset. The results on the test set show higher recall and F1-score compared to logistic regression. 

### 3. Resampling Techniques

#### Oversampling Minority Class
Oversampling can be defined as adding more copies of the minority class. Oversampling can be a good choice when you don't have a ton of data to work with. A con to consider when undersampling is that it can cause overfitting and poor generalization to your test set.

We will use the resampling module from Scikit-Learn to randomly replicate samples from the minority class.

Important Note
Always split into test and train sets BEFORE trying any resampling techniques! Oversampling before splitting the data can allow the exact same observations to be present in both the test and train sets! This can allow our model to simply memorize specific data points and cause overfitting.

In [21]:
from sklearn.utils import resample

In [22]:
# concatenate our training data back together before doing the resampling

train_set = pd.concat([X_train, y_train], axis=1)
train_set.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
13707,163892.0,1.794257,-0.277896,-2.381295,0.227035,1.181389,0.610771,0.134762,0.097202,0.646861,...,0.271430,0.845002,-0.143285,-2.082546,0.116228,0.046821,0.031267,-0.032991,104.71,0
20703,167418.0,-2.425979,2.399782,-0.779091,-1.365301,0.260095,-0.643864,1.098915,-0.142483,2.054626,...,-0.747394,-0.596796,0.001252,-0.452732,-0.161442,-0.003178,-0.337996,-1.084199,9.99,0
10572,162529.0,0.072139,1.001944,-0.401543,-0.541423,1.116271,-0.754787,0.995224,-0.139046,-0.205811,...,-0.315193,-0.734425,0.115187,0.528007,-0.388949,0.115610,0.227464,0.087912,8.99,0
4755,159856.0,2.006937,0.630784,-1.091179,3.554664,0.775172,-0.093498,0.311137,-0.177965,-1.054687,...,0.175626,0.520310,-0.026899,-0.731376,0.221499,0.177702,-0.047755,-0.060361,2.28,0
8068,161407.0,-0.004653,1.075463,-0.341932,-0.621717,0.996624,-0.781242,1.112333,-0.269529,0.355222,...,-0.422035,-0.835089,0.096160,0.553738,-0.427988,0.093356,0.167179,-0.080997,1.78,0


In [23]:
# separate minority and majority classes
train_set_not_fraud = train_set[train_set.Class==0]
train_set_fraud = train_set[train_set.Class==1]

# upsample minority
fraud_upsampled = resample(train_set_fraud,
                          replace=True, # sample with replacement
                          n_samples=train_set_not_fraud.shape[0], # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
train_set_upsampled = pd.concat([train_set_not_fraud, fraud_upsampled])

# check new class counts
train_set_upsampled.Class.value_counts()

0    21323
1    21323
Name: Class, dtype: int64

In [24]:
# trying logistic regression again with the balanced dataset
X_train_upsampled = train_set_upsampled.drop('Class', axis=1)
y_train_upsampled = train_set_upsampled.Class


upsampled = LogisticRegression(solver='liblinear').fit(X_train_upsampled, y_train_upsampled)
upsampled_pred = upsampled.predict(X_test)

print(classification_report(y_test, upsampled_pred))

              precision    recall  f1-score   support

           0       1.00      0.95      0.98      7108
           1       0.03      1.00      0.07        12

    accuracy                           0.95      7120
   macro avg       0.52      0.98      0.52      7120
weighted avg       1.00      0.95      0.97      7120



In [25]:
# confusion matrix
pd.DataFrame(confusion_matrix(y_test, upsampled_pred))

,0,1
0,6776,332
1,0,12


The accuracy score (0.97) dropped after upsampling, but the model is now predicting all the fraud instances  correctly, an improvement over our plain logistic regression above.



#### Undersampling Majority Class

Undersampling can be defined as removing some observations of the majority class. Undersampling can be a good choice when you have a lot of data - think millions of rows. But a drawback to undersampling is that we are removing information that may be valuable.

We will again use the resampling module from Scikit-Learn to randomly remove samples from the majority class.

In [26]:
# still using our separated classes fraud and not_fraud from above

# downsample majority
not_fraud_downsampled = resample(train_set_not_fraud,
                                replace = False, # sample without replacement
                                n_samples = train_set_fraud.shape[0], # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
train_set_downsampled = pd.concat([not_fraud_downsampled, train_set_fraud])

# checking counts
train_set_downsampled.Class.value_counts()

0    37
1    37
Name: Class, dtype: int64

In [27]:
# trying logistic regression again with the undersampled dataset

y_train_downsampled = train_set_downsampled.Class
X_train_downsampled = train_set_downsampled.drop('Class', axis=1)

undersampled = LogisticRegression(solver='liblinear').fit(X_train_downsampled, y_train_downsampled)
undersampled_pred = undersampled.predict(X_test)

print(classification_report(y_test, undersampled_pred))

              precision    recall  f1-score   support

           0       1.00      0.85      0.92      7108
           1       0.01      1.00      0.02        12

    accuracy                           0.85      7120
   macro avg       0.51      0.93      0.47      7120
weighted avg       1.00      0.85      0.92      7120



In [28]:
# confusion matrix
pd.DataFrame(confusion_matrix(y_test, undersampled_pred))

,0,1
0,6060,1048
1,0,12


The accuracy score (0.85) dropped after downsampling, but the model is predicting the minority class correctly although it performs worse compared to the upsampling technique.



### Generate Synthetic Samples

SMOTE or Synthetic Minority Oversampling Technique is a popular algorithm to create sythetic observations of the minority class.

In [29]:
from imblearn.over_sampling import SMOTE

In [30]:
sm = SMOTE()
X_train, y_train = sm.fit_resample(X_train, y_train)

In [31]:
smote = LogisticRegression(solver='liblinear').fit(X_train, y_train)
smote_pred = smote.predict(X_test)

In [32]:
# confustion matrix
pd.DataFrame(confusion_matrix(y_test, smote_pred))

,0,1
0,6945,163
1,0,12


In [33]:
print(classification_report(y_test, smote_pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      7108
           1       0.07      1.00      0.13        12

    accuracy                           0.98      7120
   macro avg       0.53      0.99      0.56      7120
weighted avg       1.00      0.98      0.99      7120



SMOTE performs better than upsampling and downsampling techniques.  

# Save your notebook, then `File > Close and Halt`